#### Basic Model to toxic classification

In [1]:
# importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import string
import nltk
from nltk.corpus import stopwords
from collections import Counter
stopwords = nltk.corpus.stopwords.words('english')
import difflib

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# opening a file which contains the list of volgorirty words
with open("list.txt") as f:
    lines=f.read().splitlines()

In [4]:
toxic_word=lines

In [5]:

def words(text): return re.findall(r'\w+', text.lower())
WORDS = Counter(words(open('list1.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

In [6]:
def spellCheckToxic(str):
    str_new=[]
    final_str=[]
    words=nltk.word_tokenize(str)
    for i in words:
        corrected_word=correction(i)
        a=i
        b=corrected_word
        seq = difflib.SequenceMatcher(None,a,b)
        d = seq.ratio()*100
#     print(d) 
        if(d>80):
            str_new.append(corrected_word)
        else:
            str_new.append(i)
#     print(str_new)
    str_new=' '.join(str_new)
    final_str.append(str_new)
    return final_str

In [7]:
WORDS2 = words(open('list1.txt').read())

In [8]:
from autocorrect import spell
def spell_check(text):
    f=[]
    li=nltk.word_tokenize(text)
    for k in li:
        toxic_word = k
        j=0
        def words(text): return re.findall(r'\w+', text.lower())
        count = 0

        
        for i in WORDS2:
            if i == toxic_word:
                count = 2
        l=spellCheckToxic(toxic_word)
        tc = 0
        for i in WORDS2:
            if i == l[0]:
                tc=1
                count = count + 1
                break
        toxic_word2 = spell(toxic_word)
        nc = 0
        for i in WORDS2:
            if i == toxic_word2:
                nc=1
                count += 3
                break
        if count >= 3:
            j =1
        else:
             if(toxic_word == toxic_word2 or l[0] == toxic_word):
                j=1
        
             else:
                 j=0
        if(j==1):
            m=spell(k)
            f.append(m)
        else:
            m=spellCheckToxic(k)
            f.append(m[0])
    final_text=" ".join(f)
    return final_text

### Stage 1: Predicting toxic with Bag_of_word (Rule based Method)

In [9]:

from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
stemmer=SnowballStemmer('english')

In [10]:
def Cleaning_data(line):
    line=spell_check(line)
    line=re.sub(r"(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*","",line)
        # remove the unnecessary punctuations
    line=re.sub(r"[^a-zA-Z ]","",line)
    line=re.sub(r'(\w)\1{2,}',r'\1',line)

        # remove number
    line=re.sub(r"[0-9]","",line)
    
    tokens_sent=word_tokenize(line)
    
        
    return tokens_sent

In [11]:
# stage 1 Bag of words
def Bag_of_Words(data):
    line=set(Cleaning_data(data))
    
    toxic=False
    count=0
    for word in line:
        if word in toxic_word:
            count+=1
    if(count>0):
        toxic=True
    return toxic
        
    
    

In [12]:
text="words"

In [13]:
k=Bag_of_Words(text)

autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead


In [14]:
k

False

### Machine learning Model for Toxic comment classification

In [15]:
# Reading  the datasets 
train_set=pd.read_csv("train.csv")
test_set=pd.read_csv("test.csv")
test_set_labels=pd.read_csv("test_labels.csv")

In [16]:
# converting the all categorical columsn in to one colume i.e toxic,serve,insult,obsecen,threat=== toxic
index=train_set.loc[(train_set.toxic==1)|(train_set.severe_toxic==1)|(train_set.obscene==1)|(train_set.threat==1)|(train_set.insult==1)|(train_set.identity_hate==1)].index
new_column=list(train_set.toxic)
for i in index:
    new_column[i]=1

In [17]:
# Reading the alredy Preprocessed Data which was steeming and removed the stopword
preprocessed_data=pd.read_csv("processed.csv")
preprocessed_data["toxic"]=new_column

In [18]:
# displaying the data set which is Preprocessed 
display(preprocessed_data.head())

,comment_text,toxic
0,explan edit made usernam hardcor metallica fan...,0
1,d'aww ! match background colour 'm seem stuck ...,0
2,"hey man , 'm realli tri edit war . 's guy cons...",0
3,`` ca n't make real suggest improv - wonder se...,0
4,", sir , hero . chanc rememb page 's ?",0


#### Removing the HTML tags and unnecssary punctuation marks

In [19]:
## cleaning the data 
## removing the html tags and the some unimportant punctuations
def CCleaning_data(line):
    line=re.sub(r"(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*","",line)
        # remove the unnecessary punctuations
    line=re.sub(r"[^a-zA-Z ]","",line)

        # remove numbers
    line=re.sub(r"[0-9]","",line)
    return line

In [20]:
clean_data=[]
for sent in preprocessed_data["comment_text"]:
    clean_data.append(CCleaning_data(sent))

In [21]:
# Repacing the comment_text with the clean data
preprocessed_data["comment_text"]=clean_data

In [22]:
# dispalying the preprocessed dataa
display(preprocessed_data.head())

,comment_text,toxic
0,explan edit made usernam hardcor metallica fan...,0
1,daww match background colour m seem stuck th...,0
2,hey man m realli tri edit war s guy constant...,0
3,ca nt make real suggest improv wonder sectio...,0
4,sir hero chanc rememb page s,0


#### Splitting the data in to the training and testing data

In [23]:
from sklearn.cross_validation import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(preprocessed_data["comment_text"],preprocessed_data["toxic"],test_size=0.2)

In [24]:
##  size of the training and testing dataset
print("the size of the training_set :",X_train.shape,Y_train.shape)
print("the size of the testing_set  :",X_test.shape,Y_test.shape)

the size of the training_set : (127656,) (127656,)
the size of the testing_set  : (31915,) (31915,)


In [25]:
preprocessed_data["toxic"].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

### Converting the document corpus to the vector

In [26]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer

In [27]:
# using the TF_ID vectorization and for anlayzer is word
def convert_text_to_vector():
    word_vec=TfidfVectorizer(sublinear_tf=True,analyzer='word',stop_words='english',lowercase=True,token_pattern=r'\w{3,}',strip_accents='unicode',ngram_range=(1,2),max_features=30000)
    char_vec=TfidfVectorizer(analyzer='char',max_features=1000,ngram_range=(2,4),sublinear_tf=True,stop_words='english',strip_accents='unicode')
    return word_vec,char_vec

In [28]:
word_vctorizer,char_vectorizer=convert_text_to_vector()

In [29]:
## fitting the preprocessed data in to the vectorizer
training_features=word_vctorizer.fit_transform(X_train)

In [30]:
testing_features=word_vctorizer.transform(X_test)

## Building the Machine learning Models

- Features data : {training_features, testing_features}

#### 1) LOGISTIC REGRESSION

In [31]:
from sklearn.linear_model import LogisticRegression

In [32]:
# Logistic Regression Model
Log_classifer=LogisticRegression(C=1.4,solver='sag',max_iter=500)
#fit the data
Log_classifer.fit(training_features,Y_train)

LogisticRegression(C=1.4, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False)

In [33]:
# Predicting thelabels for the training data
log_train_pre=Log_classifer.predict(training_features)

In [34]:
# Predicting the labels for the testing data
log_test_pre=Log_classifer.predict(testing_features)

#### Finding the accuracy for both train and testing data comapre for overfitting

In [35]:
from sklearn.metrics import f1_score,accuracy_score,matthews_corrcoef,roc_auc_score

In [36]:
# f1_score accuracy
print("--------------F1_SCORE------------------------")
print()
print("Testing accuracy is :----->",f1_score(Y_train,log_train_pre))
# testing accuracy 
print("Testing accuracy is :------>",f1_score(Y_test,log_test_pre))
print()
# matthews_corrcoef
print("-------------MATTHEWS_CORRCOEF_SCORE------------------------")
print()
print("Testing accuracy is :----->",matthews_corrcoef(Y_train,log_train_pre))
# testing accuracy 
print("Testing accuracy is :------>",matthews_corrcoef(Y_test,log_test_pre))
print()
#roc_auc_score
print("--------------ROC_AUC_SCORE------------------------")
print()
print("Testing accuracy is :----->",roc_auc_score(Y_train,log_train_pre))
# testing accuracy 
print("Testing accuracy is :------>",roc_auc_score(Y_test,log_test_pre))
print()
#accuracy score
print("--------------ACCURACY_SCORE------------------------")
print()
print("Testing accuracy is :----->",accuracy_score(Y_train,log_train_pre))
# testing accuracy 
print("Testing accuracy is :------>",accuracy_score(Y_test,log_test_pre))

--------------F1_SCORE------------------------

Testing accuracy is :-----> 0.7948511665325825
Testing accuracy is :------> 0.7608734885399747

-------------MATTHEWS_CORRCOEF_SCORE------------------------

Testing accuracy is :-----> 0.7877471323673234
Testing accuracy is :------> 0.7527513717655351

--------------ROC_AUC_SCORE------------------------

Testing accuracy is :-----> 0.8405588745004278
Testing accuracy is :------> 0.8207041504668859

--------------ACCURACY_SCORE------------------------

Testing accuracy is :-----> 0.9640439932318104
Testing accuracy is :------> 0.958483471721761


###  .

#### 2)RIDGE CLASSIFIER

In [37]:
from sklearn.linear_model import RidgeClassifier

In [38]:
Ridge_classifier=RidgeClassifier(alpha=2,solver='sag')

In [39]:
Ridge_classifier.fit(training_features,Y_train)

RidgeClassifier(alpha=2, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='sag',
        tol=0.001)

In [40]:
# prediction on the train set 
ridge_train_pre=Ridge_classifier.predict(training_features)
# Prediction on the test set
ridge_test_pre=Ridge_classifier.predict(testing_features)

In [41]:
# f1_score accuracy
print("--------------F1_SCORE------------------------")
print()
print("Testing accuracy is :----->",f1_score(Y_train,ridge_train_pre))
# testing accuracy 
print("Testing accuracy is :------>",f1_score(Y_test,ridge_test_pre))
print()
# matthews_corrcoef
print("-------------MATTHEWS_CORRCOEF_SCORE------------------------")
print()
print("Testing accuracy is :----->",matthews_corrcoef(Y_train,ridge_train_pre))
# testing accuracy 
print("Testing accuracy is :------>",matthews_corrcoef(Y_test,ridge_test_pre))
print()
#roc_auc_score
print("--------------ROC_AUC_SCORE------------------------")
print()
print("Testing accuracy is :----->",roc_auc_score(Y_train,ridge_train_pre))
# testing accuracy 
print("Testing accuracy is :------>",roc_auc_score(Y_test,ridge_test_pre))
print()
#accuracy score
print("--------------ACCURACY_SCORE------------------------")
print()
print("Testing accuracy is :----->",accuracy_score(Y_train,ridge_train_pre))
# testing accuracy 
print("Testing accuracy is :------>",accuracy_score(Y_test,ridge_test_pre))

--------------F1_SCORE------------------------

Testing accuracy is :-----> 0.7705895953757227
Testing accuracy is :------> 0.713063320022818

-------------MATTHEWS_CORRCOEF_SCORE------------------------

Testing accuracy is :-----> 0.7686302063045947
Testing accuracy is :------> 0.7130052347867653

--------------ROC_AUC_SCORE------------------------

Testing accuracy is :-----> 0.8197945828313818
Testing accuracy is :------> 0.7857679098021939

--------------ACCURACY_SCORE------------------------

Testing accuracy is :-----> 0.9611377451901987
Testing accuracy is :------> 0.9527181576061413


### .

#### NAIVE BAYES MULTINOMIAL MODEL

In [42]:
from sklearn.naive_bayes import MultinomialNB

In [43]:
naive_classifer=MultinomialNB(alpha=0.1)
naive_classifer.fit(training_features,Y_train)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [44]:
# predicting the training and testing features
naive_train_pre=naive_classifer.predict(training_features)
naive_test_pre=naive_classifer.predict(testing_features)

In [45]:
# f1_score accuracy
print("--------------F1_SCORE------------------------")
print()
print("Testing accuracy is :----->",f1_score(Y_train,naive_train_pre))
# testing accuracy 
print("Testing accuracy is :------>",f1_score(Y_test,naive_test_pre))
print()
# matthews_corrcoef
print("-------------MATTHEWS_CORRCOEF_SCORE------------------------")
print()
print("Testing accuracy is :----->",matthews_corrcoef(Y_train,naive_train_pre))
# testing accuracy 
print("Testing accuracy is :------>",matthews_corrcoef(Y_test,naive_test_pre))
print()
#roc_auc_score
print("--------------ROC_AUC_SCORE------------------------")
print()
print("Testing accuracy is :----->",roc_auc_score(Y_train,naive_train_pre))
# testing accuracy 
print("Testing accuracy is :------>",roc_auc_score(Y_test,naive_test_pre))
print()
#accuracy score
print("--------------ACCURACY_SCORE------------------------")
print()
print("Testing accuracy is :----->",accuracy_score(Y_train,naive_train_pre))
# testing accuracy 
print("Testing accuracy is :------>",accuracy_score(Y_test,naive_test_pre))

--------------F1_SCORE------------------------

Testing accuracy is :-----> 0.7682231331313492
Testing accuracy is :------> 0.7087994034302758

-------------MATTHEWS_CORRCOEF_SCORE------------------------

Testing accuracy is :-----> 0.7575537550709208
Testing accuracy is :------> 0.7026255078551547

--------------ROC_AUC_SCORE------------------------

Testing accuracy is :-----> 0.8286250207414912
Testing accuracy is :------> 0.7883835379141865

--------------ACCURACY_SCORE------------------------

Testing accuracy is :-----> 0.9592498589960519
Testing accuracy is :------> 0.9510574964750117


###### .

#### XG BOOST MODEL

In [46]:
import xgboost as xgb

In [47]:
xgb_classifier=xgb.XGBClassifier(random_state=1,learning_rate=0.15,max_depth=5,n_estimators=300)

In [48]:
xgb_classifier.fit(training_features,Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.15,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=300, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=1, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [49]:
# predicting the training and testing features
xgb_train_pre=xgb_classifier.predict(training_features)
xgb_test_pre=xgb_classifier.predict(testing_features)

In [50]:
# f1_score accuracy
print("--------------F1_SCORE------------------------")
print()
print("Testing accuracy is :----->",f1_score(Y_train,xgb_train_pre))
# testing accuracy 
print("Testing accuracy is :------>",f1_score(Y_test,xgb_test_pre))
print()
# matthews_corrcoef
print("-------------MATTHEWS_CORRCOEF_SCORE------------------------")
print()
print("Testing accuracy is :----->",matthews_corrcoef(Y_train,xgb_train_pre))
# testing accuracy 
print("Testing accuracy is :------>",matthews_corrcoef(Y_test,xgb_test_pre))
print()
#roc_auc_score
print("--------------ROC_AUC_SCORE------------------------")
print()
print("Testing accuracy is :----->",roc_auc_score(Y_train,xgb_train_pre))
# testing accuracy 
print("Testing accuracy is :------>",roc_auc_score(Y_test,xgb_test_pre))
print()
#accuracy score
print("--------------ACCURACY_SCORE------------------------")
print()
print("Testing accuracy is :----->",accuracy_score(Y_train,xgb_train_pre))
# testing accuracy 
print("Testing accuracy is :------>",accuracy_score(Y_test,xgb_test_pre))

--------------F1_SCORE------------------------

Testing accuracy is :-----> 0.7866539214347489
Testing accuracy is :------> 0.7471807930156421

-------------MATTHEWS_CORRCOEF_SCORE------------------------

Testing accuracy is :-----> 0.7826467348716458
Testing accuracy is :------> 0.7392860682741066

--------------ROC_AUC_SCORE------------------------

Testing accuracy is :-----> 0.8315540971371518
Testing accuracy is :------> 0.812217314497102

--------------ACCURACY_SCORE------------------------

Testing accuracy is :-----> 0.9632841386225481
Testing accuracy is :------> 0.9564468118439605


###### .

#### SAVING THE MODELS

In [51]:
import pickle

In [52]:
# saving the logistic regression model (model : classifier)
pickle.dump(Log_classifer,open("toxic_logistic.sav",'wb'))
pickle.dump(Ridge_classifier,open("toxic_Ridge.sav",'wb'))
pickle.dump(naive_classifer,open("toxic_Naive.sav",'wb'))
pickle.dump(xgb_classifier,open("toxic_xgb.sav",'wb'))
# saving the vectorizer
pickle.dump(word_vctorizer,open("vectorizer.sav",'wb'))

###### .

#### Testing the User comments

In [53]:
# Preprocess the user comments
def EG_Preprocess(line):
    line=re.sub(r"(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*","",line)
    # remove the unnecessary punctuations
    line=re.sub(r"[^,.?a-zA-Z! ]","",line)
    # removing the repeted letters
    line=re.sub(r'(\w)\1{2,}',r'\1',line)
       
    # remove numbers
    line=re.sub(r"[0-9]","",line)
    # Tokienization of the sentence   
    tokens_sent=word_tokenize(line)
    filt_sent=[]
    for word in tokens_sent:
        if(word not in stopwords):
            filt_sent.append(word)
        #stemming
        stemmed_sent=[stemmer.stem(x) for x in filt_sent]
        # converting the words in to the text
    sentt=' '.join(stemmed_sent)
    return sentt

In [54]:
def Testing_comments(data):
    li=[]
    for line in data:
        sent=EG_Preprocess(line)
        li.append(sent)
    

    return li

In [55]:
def Machine_Predictions(data):
    data=Testing_comments(data)
    test_vector=word_vctorizer.transform(data)
    l=Log_classifer.predict(test_vector)
    r=Ridge_classifier.predict(test_vector)
    n=naive_classifer.predict(test_vector)
    x=xgb_classifier.predict(test_vector)
    dic={"Logistic":l,
         "Ridge":r,
        "Naive":n,
         "XGB":x }
    return dic
def Machine_Predictions_Probs(data):
    data=Testing_comments(data)
    test_vector=word_vctorizer.transform(data)
    l=Log_classifer.predict_proba(test_vector)[:,1]
    r=Ridge_classifier.predict(test_vector)
    n=naive_classifer.predict_proba(test_vector)[:,1]
    x=xgb_classifier.predict_proba(test_vector)[:,1]
    dic={"Logistic":l,
         "Ridge":r,
        "Naive":n,
         "XGB":x }
    return dic

In [56]:
text=["these fellows are"]

In [57]:
a=Machine_Predictions(text)

In [107]:
def mac_toxic(txt):
    toxic=False
    d=Machine_Predictions(txt)
    
    if(d["Logistic"]==1 or d["Ridge"]==1 or d["Naive"]==1 or d["XGB"]==1):
        toxic=True
    return toxic
        

##### .

#### Applying the TWO stages Bag of Words and Machine learing Model

In [219]:
user_test=["idiot"]

In [220]:
def Analysis(text):
    if(Bag_of_Words(text[0]) or mac_toxic(text)):
        return True
    else:
        return False
    

In [221]:
a=Analysis(user_test)

autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead


In [222]:
a

True

In [214]:
Machine_Predictions_Probs(user_test)

{'Logistic': array([0.94670324]),
 'Ridge': array([1]),
 'Naive': array([0.73620973]),
 'XGB': array([0.4761731], dtype=float32)}

In [168]:
mac_toxic(user_test)

True

In [169]:
Machine_Predictions(user_test)

{'Logistic': array([1]),
 'Ridge': array([1]),
 'Naive': array([1]),
 'XGB': array([1])}

In [1]:
import pandas as pd